In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from src.data import Data
from src.plothelp import *
import time

### Load and pre-process the dataset:


In [3]:
data = Data("img")
data.load_pickle("100x100.pickle ")
data.resize_img((56,56)) 
X_train, X_test, y_train, y_test = data.train_test_split()

In [4]:
#Convert python lists to np arrays
X_trains = np.asarray(X_train)
X_tests = np.asarray(X_test)
y_trains = np.asarray(y_train)
y_tests = np.asarray(y_test)
print(X_trains.shape, X_tests.shape, y_trains.shape, y_tests.shape)

(4698, 56, 56, 3) (522, 56, 56, 3) (4698,) (522,)


In [5]:
y_oh, y_test_oh= data.one_hot(y_trains, y_tests)

In [6]:
# Split y_train into 54 batches containing 87 img
y_batch = np.array_split(y_oh, 54 )
y_batch = np.asarray(y_batch)
print(y_batch.shape)

(54, 87, 4)


In [7]:
# Split X_test into 54 batches of 87 img
X_train_batch = np.array_split(X_trains, 54 )
X_train_batch = np.asarray(X_train_batch)
print(X_train_batch.shape)

(54, 87, 56, 56, 3)


In [8]:
learning_rate = 0.001
batch_size = 87
n_epochs = 100
keep_prob  = 0.8

In [9]:
# input X: 28x28 grayscale images
X = tf.placeholder(tf.float32, [None, 56, 56, 3])
Y = tf.placeholder(tf.float32, [None, 4])

In [10]:
l1 = 4  # first convolutional layer output depth
l2 = 8  # second convolutional layer output depth
l3 = 12  # third convolutional layer
l4 = 12  # forth conv
l5 = 200  # fully connected layer
num_classes = 4 

In [11]:
w1 = tf.Variable(tf.truncated_normal([5, 5, 3, l1], stddev=0.1))  # 5x5 patch, 3 input channel, K output channels
b1 = tf.Variable(tf.random_normal([l1]))

w2 = tf.Variable(tf.truncated_normal([5, 5, l1, l2], stddev=0.1))
b2 = tf.Variable(tf.random_normal([l2]))

w3 = tf.Variable(tf.truncated_normal([4, 4, l2, l3], stddev=0.1))
b3 = tf.Variable(tf.random_normal([l3]))

w4 = tf.Variable(tf.truncated_normal([4, 4, l3,l4], stddev=0.1))
b4 = tf.Variable(tf.random_normal([l4]))

w5 = tf.Variable(tf.truncated_normal([7 * 7 * l4, l5], stddev=0.1))
b5 = tf.Variable(tf.random_normal([l5]))

w = tf.Variable(tf.truncated_normal([l5, num_classes], stddev=0.1))
b = tf.Variable(tf.random_normal([num_classes]))

In [12]:
# The model
Y1 = tf.nn.relu(tf.nn.conv2d(X, w1, strides=[1, 1, 1, 1], padding='SAME') + b1)
Y1 = tf.nn.dropout(Y1,keep_prob=keep_prob) #for adding dropout, comment to disable

Y2 = tf.nn.conv2d(Y1, w2, strides=[1, 1, 1, 1], padding='SAME')
Y2 = tf.nn.max_pool(Y2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')  # reduce to 28x28
Y2 = tf.nn.relu( Y2+ b2)
Y2 = tf.nn.dropout(Y2,keep_prob=keep_prob) #for adding dropout, comment to disable

Y3 = tf.nn.conv2d(Y2, w3, strides=[1, 1, 1, 1], padding='SAME')
Y3 = tf.nn.max_pool(Y3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')  # reduce to 14x14
Y3 = tf.nn.relu(Y3+ b3)
Y3 = tf.nn.dropout(Y3,keep_prob=keep_prob) #for adding dropout, comment to disable

Y4 = tf.nn.conv2d(Y3, w4, strides=[1, 1, 1, 1], padding='SAME')
Y4 = tf.nn.max_pool(Y3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')  # reduce to 7x7
Y4 = tf.nn.relu(Y4+ b4)
Y4 = tf.nn.dropout(Y4,keep_prob=keep_prob)

# fully connected layer
fc = tf.reshape(Y4, shape=[-1, 7 * 7 * l4])

Y5 = tf.nn.relu(tf.matmul(fc, w5) + b5)
Y5 = tf.nn.dropout(Y5,keep_prob=keep_prob)
Ylogits = tf.matmul(Y5, w) + b
logits = tf.nn.softmax(Ylogits)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy)

# Calculate the accuracy of the model
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [13]:

init = tf.global_variables_initializer()

In [14]:
with tf.Session() as sess:
    sess.run(init)
    n_batches = 54
    for epochs in range(100):
        acc = 0
        for i in range(n_batches):
            batch_X = X_train_batch[i, :]
            batch_Y = y_batch[i, :]
            _, a = sess.run([train_step, accuracy], {X: batch_X, Y: batch_Y})
            acc=acc+a
        print("Epoch:", '%04d' % (epochs + 1), "accuracy=", "{:.9f}".format(acc/54))

    saver = tf.train.Saver()
    save_path = saver.save(sess, "./models/conv56x56x3-drop-100.ckpt")
    print("Model saved in file: %s" % save_path)

Epoch: 0001 accuracy= 0.282673479
Epoch: 0002 accuracy= 0.276926354
Epoch: 0003 accuracy= 0.288846318
Epoch: 0004 accuracy= 0.296509154
Epoch: 0005 accuracy= 0.347807579
Epoch: 0006 accuracy= 0.371647508
Epoch: 0007 accuracy= 0.379736058
Epoch: 0008 accuracy= 0.395274587
Epoch: 0009 accuracy= 0.394210302
Epoch: 0010 accuracy= 0.398254576
Epoch: 0011 accuracy= 0.412515964
Epoch: 0012 accuracy= 0.415283098
Epoch: 0013 accuracy= 0.431673052
Epoch: 0014 accuracy= 0.422307365
Epoch: 0015 accuracy= 0.426777352
Epoch: 0016 accuracy= 0.438484460
Epoch: 0017 accuracy= 0.439974457
Epoch: 0018 accuracy= 0.458492976
Epoch: 0019 accuracy= 0.475521496
Epoch: 0020 accuracy= 0.473180077
Epoch: 0021 accuracy= 0.473392930
Epoch: 0022 accuracy= 0.481268622
Epoch: 0023 accuracy= 0.485738608
Epoch: 0024 accuracy= 0.494678585
Epoch: 0025 accuracy= 0.493614299
Epoch: 0026 accuracy= 0.509578539
Epoch: 0027 accuracy= 0.522775649
Epoch: 0028 accuracy= 0.515325667
Epoch: 0029 accuracy= 0.516602805
Epoch: 0030 ac

In [17]:
with tf.Session() as sess:
    sess.run(init)
    saver = tf.train.Saver().restore(sess, "./models/conv56x56x3-drop-100.ckpt")
    print("Model restored.")
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={X: X_tests, Y: y_test_oh}))

Model restored.
0.693487
